In [1]:
import spacy
import re
import pandas as pd
from keras.datasets import imdb
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import os
import spacy
from spacy.util import minibatch, compounding
from nltk import word_tokenize
import progressbar
import numpy as np
from tqdm import tqdm
from nltk.parse.stanford import StanfordDependencyParser

/Users/dmlitov4/Documents/machineLearning/mlenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tqdm.pandas()

In [3]:
nlp = spacy.load('spacy_models/')

OSError: [E050] Can't find model 'spacy_models/'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [4]:
paths = [dir for dir,_,_ in os.walk('data/storage/storage/') ][1:]

In [5]:
projects = pd.read_csv('data/projects_final.csv',sep = ';')

In [6]:
tweets = pd.read_csv('data/tweets_final.csv',sep=';')

In [7]:
tweets.shape

(298876, 6)

In [8]:
tweets.head()

,id,source_id,author,source_message_id,publish_date,text
0,1,1,Excellion,1026686499805249536,2018-08-07 04:28:31,@johnkim77 @SatoshiLite @bensemchee Chikun &am...
1,2,1,Excellion,1026685414373974016,2018-08-07 04:24:12,@brockm He’s talking to you!
2,3,1,Excellion,1026638290374414337,2018-08-07 01:16:57,@ptyx11 @EmbassyPty @fluffypony @Chainzillaio ...
3,4,1,Excellion,1026631769137733632,2018-08-07 00:51:02,"RT @notgrubles: Remember, ""Bcash"" originated n..."
4,5,1,Excellion,1026629513612939264,2018-08-07 00:42:04,RT @infinitefleet: The countdown has begun! Ou...


In [9]:
def clean_tweet(text):
    return re.sub(pattern='https.+|\n|\\\\n|\"|^\d.\/|@\w+|\W',repl=' ',string=text).strip()

In [10]:
def get_sentiment(text,spacy_model = nlp,threshold = 0.5):
    prob = spacy_model(text).cats['POSITIVE']
    if prob>threshold+0.1:
        return 1
    elif prob<threshold-0.1:
        return -1
    else:
        return 0

In [11]:
def check_mention(tweet, project):
    pt = project.lower()
    tt = word_tokenize(tweet.lower())
    if pt in tt:
        return 1
    else:
        return 0

In [13]:
path_to_jar = './stanford-corenlp-full-2018-02-27/stanford-corenlp-3.9.1.jar'
path_to_models_jar = './stanford-corenlp-full-2018-02-27/stanford-corenlp-3.9.1-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)
def check_for_subject(term,sentence):
    result = dependency_parser.raw_parse(sentence)
    dep = [r.triples() for r in result][0]
    trips = [d for d in dep]
    result = np.sum([1 for t in trips if 'nsubj' in str(t) and term in str(t)])
    return result

In [14]:
def get_mentions(project,df):
    df['mentions'] = df.progress_apply(lambda x: check_mention(x.text,project),axis = 1)
    if df[df.mentions>0].text.shape[0]>0:
        return df[df.mentions>0]
    else:
        return np.array([])

In [15]:
def get_subjects(project,df):
    df['subject'] = df.progress_apply(lambda x: check_for_subject(project, x.text),axis = 1)
    if df[df.subject>0].text.shape[0]>0:
        return df[df.subject>0]
    else:
        return np.array([])

In [16]:
tweets['cleaned'] = tweets.text.apply(clean_tweet)

In [17]:
tweets = tweets[tweets.cleaned.progress_apply(word_tokenize).apply(len)>2]

100%|██████████| 298876/298876 [00:37<00:00, 7966.55it/s]


In [17]:
projects_list = projects.name.tolist()

In [24]:
res = []
mens = []
for p in progressbar.progressbar(projects_list):
    m = get_mentions(p,df=tweets)
    mens.append(m.shape[0])
    if m.shape[0]>0:
        s = m.cleaned.apply(get_sentiment).sum()
        res.append(s)
    else:
        res.append(None)

100% (911 of 911) |######################| Elapsed Time: 2:42:58 Time:  2:42:58


In [18]:
r = pd.DataFrame({'score': res,'mentions': mens},index = projects_list.tolist())

NameError: name 'res' is not defined

In [39]:
r.loc['ltc',:]

score       -81
mentions    569
Name: ltc, dtype: int64

In [218]:
r.to_csv('60projects.csv')

In [19]:
rabbit = 'eth'

In [20]:
rdf = get_mentions(df=tweets.sample(frac = 1),project=rabbit)

100%|██████████| 279064/279064 [01:20<00:00, 3486.37it/s]


In [21]:
rdf.shape

(1872, 8)

In [ ]:
rsdf = get_subjects(df=rdf,project=rabbit)

 29%|██▉       | 542/1872 [17:23<52:44,  2.38s/it]  

In [ ]:
rsdf.shape

In [ ]:
rdf = rdf[rdf.cleaned.apply(word_tokenize).apply(len)>1]

In [33]:
rsdf['sent'] = rsdf.sample(frac = 1.0).cleaned.progress_apply(get_sentiment)

100%|██████████| 8/8 [00:00<00:00, 193.31it/s]
/home/lsm/anaconda3/envs/ipavlov/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
rsdf.sent.value_counts()

-1    3
 1    3
 0    2
Name: sent, dtype: int64

In [36]:
rsdf.sent.sum()

0

In [41]:
rsdf[['cleaned','sent']]

,cleaned,sent
150018,Yes that s empirically true Note that I said...,1
165348,I forgot that people pronounce ETH as eeth ...,0
66287,RT sold all my eth for some more etc Ho...,1
124733,RT maybe this is what the ethereum flip...,-1
291331,Oh look something is worth 0 10 it will be w...,-1
209175,Bitcoin Core is reference client for bitcoin ...,-1
250758,RT ETH will do one of two planned hard for...,0
43385,Unsplit ETH is not ETC Unsplit ether is ET...,1


In [45]:
rsdf.cleaned.iloc[2]

'RT    sold all my  eth for some more  etc   Holders will follow'